# Structured Output

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

# Setup Tracing

In [8]:
from langfuse.openai import AsyncOpenAI

client = AsyncOpenAI()

GPT_4O_MINI = "gpt-4o-mini"

# Chat Helpers

In [9]:
def _msg(role: str, content: str) -> dict[str, str]:
    return {"role": role, "content": content}


def _system(content: str) -> dict[str, str]:
    return _msg(role="system", content=content)


def _user(content: str) -> dict[str, str]:
    return _msg(role="user", content=content)


def _assistant(content: str) -> dict[str, str]:
    return _msg(role="assistant", content=content)

In [10]:
from pydantic import BaseModel
from openai import OpenAI


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


completion = await client.beta.chat.completions.parse(
    model=GPT_4O_MINI,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [11]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [17]:
event.model_dump()

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

In [16]:
event.model_dump_json()

'{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}'

In [18]:
completion

ParsedChatCompletion[CalendarEvent](id='chatcmpl-AXQBwzgP3BWEBtWZd0Ncj2gcT4Y4n', choices=[ParsedChoice[CalendarEvent](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])))], created=1732528868, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=17, prompt_tokens=92, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [22]:
completion.choices[0].message.parsed

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [24]:
completion.choices[0].message.content

'{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}'

In [25]:
json_string = completion.choices[0].message.content
print(json_string)

{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}


In [26]:
import json

completion_as_dict = json.loads(json_string)
completion_as_dict

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

In [27]:
CalendarEvent(**completion_as_dict)

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [32]:
event = CalendarEvent.model_validate(obj=completion_as_dict)
print(event)
print(event.name)
print(event.date)
print(event.participants)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']
Science Fair
Friday
['Alice', 'Bob']
